# Loading data set
Load datasets and encode categorical atributes

In [34]:
#encoding:utf-8
import pandas as pd
import numpy as np
from sklearn.impute import SimpleImputer

In [35]:
data = pd.read_csv('./data/use-of-force.csv', sep=",").set_index('ID');

# Tratamento dos dados faltantes

Algumas colunas do dataset possuem dados faltantes (missing data). Elas aparecem na coluna das delegacias, dos setores, dos beats, da raça e do gênero. Todas as colunas com missing data são categóricas.

Dados faltantes podem ser aleatórios ou não aleatórios. Dizemos que o dado é faltante não aleatório quando o fato da informação não estar preenchida é, em si, relevante. Por exemplo, Imagine que os policiais de uma determinada delegacia estivessem evitando de preencher o campo "Precinct" para ficar com uma estatística mais favorável. Nesse caso, o fato do campo "Precinct" não estar preenchido seria um indicio de que a ocorrência foi naquela delegacia. 

Apenas analisando os dados, não é possível afirmar se os dados faltantes são aleatórios ou não. Porém, eles estão presentes em uma quantidade significativa dos registros. Considerando que a amostra que possuimos é representativa, e em machine learning nós sempre fazemos essa suposição, então o nosso classificador será submetido ao trabalho de classificar dados faltantes com frequencia não despresível.

## Estratégia adotada

Neste trabalho, consideraremos que a ausência dos dados é uma nova classe de valores possíveis para aquele atribulto. Como em uma das classes, alguns registros possuem o valor **'-'** informado, adotaremos esse padrão para todos. Assim, o campo gênero, por exemplo, poderá ter os valores: **Masculino**, **Feminino** e **-**. 

In [36]:
imp = SimpleImputer(strategy="constant", fill_value='-');
data = imp.fit_transform(data);
[N, D] = np.shape(data);

In [37]:
# Separate data: input and target
T = data[:,1];
X = data[:,2:];

# Separete data in three classes

In [3]:
from sklearn.neighbors import KNeighborsClassifier

def rescue_condensed(X, Y):
#     knn = KNeighborsClassifier(np.floor(5*np.log10(len(data))).astype('int'))
    knn = KNeighborsClassifier(np.floor(np.sqrt(len(data))).astype('int'))
    knn.fit(X, Y)
    out = knn.predict(X)

    # get those in the overlap region
    selection = out != Y

    # yield those in the overlap region
#     return X[selection, :], Y[selection, :]
    return selection

In [4]:
fuzzySelection = rescue_condensed(X, y)
y[fuzzySelection] = 0
print('good:\t %d'%(np.sum(y==-1)))
print('fuzzy:\t %d'%(np.sum(y==0)))
print('bad:\t %d'%(np.sum(y==+1)))

good:	 687
fuzzy:	 287
bad:	 26


In [5]:
# fuzzySelection = rescue_condensed(X, y)
# X = X.loc[~fuzzySelection]
# y = y.loc[~fuzzySelection]

In [6]:
# y = y.apply(str)
# oneHot = pd.get_dummies(y)
# y = pd.concat([y,oneHot],axis=1).drop('Label',axis=1)

# Run Classifier

In [7]:
from sklearn.discriminant_analysis import QuadraticDiscriminantAnalysis
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score

cls = QuadraticDiscriminantAnalysis()
nRuns = 20
result = {'acc':0.0,'good':0.0,'fuzzy':0.0,'bad':0.0}
for r in range(nRuns):
    Xl, Xt, yl, yt = train_test_split(X, y, test_size=0.2)

    cls.fit(Xl, yl)    
    yh = []
    for i in range(len(yt)):
        xi = Xt.iloc[i].values.reshape(1, -1)     
        yh.append(cls.predict(xi)[0])  
    cmat = confusion_matrix(yt, yh)  
    
    D = cmat.diagonal()
    S = cmat.sum(axis=1)
    
    result['good'] += (D[0].astype('float')/S[0])/nRuns
    result['fuzzy'] += (D[1].astype('float')/S[1])/nRuns
    result['bad'] += (D[2].astype('float')/S[2])/nRuns
    result['acc'] += accuracy_score(yt, yh)/nRuns    
print(result)    

{'acc': 0.7022499999999999, 'fuzzy': 0.37331056541826113, 'bad': 0.5485119047619047, 'good': 0.8476800889900252}
